In [2]:
import pandas as pd
import pickle as pkl
import os

In [7]:
def get_target_df(ann_folder):
    rows = {}
    for filename in os.listdir(ann_folder):
        with open(os.path.join(ann_folder, filename), 'rb') as f:
            d = pkl.load(f)
            instance_data = next(iter(d.values()))
            anns = instance_data['diagnosis_anns']
            anns = {(k, question): v2 for k, v in anns.items() for question, v2 in v.items()}
            del instance_data['diagnosis_anns']
            instance_data.update(anns)
            rows.update(d)
    return pd.DataFrame(rows).transpose()

def get_evidence_df(ann_folder):
    rows = {}
    for filename in os.listdir(ann_folder):
        with open(os.path.join(ann_folder, filename), 'rb') as f:
            d = pkl.load(f)
            rows.update(d)
    return pd.DataFrame(rows).transpose()



In [8]:
df1 = get_target_df('/work/frink/mcinerney.de/ehr-diagnosis-annotations/annotations_9-7-23/target_annotations/val2/Andrea_Young/')
df1

,instance_idx,split,"(pneumonia, is_confident_diagnosis)","(pneumonia, could_be_identified_earlier)","(cancer, is_confident_diagnosis)","(cancer, could_be_identified_earlier)","(pulmonary edema, is_confident_diagnosis)","(pulmonary edema, could_be_identified_earlier)"
0,11,val2,No,NaN,NaN,NaN,NaN,NaN
1,12,val2,Yes,Yes,NaN,NaN,NaN,NaN
2,16,val2,NaN,NaN,Yes,Yes,NaN,NaN
3,17,val2,No,NaN,NaN,NaN,Yes,No
4,22,val2,Yes,Yes,Yes,Yes,NaN,NaN
5,22,val2,No,NaN,Yes,Yes,NaN,NaN
6,34,val2,No,NaN,NaN,NaN,Yes,No
7,35,val2,NaN,NaN,Yes,Yes,NaN,NaN
8,40,val2,NaN,NaN,NaN,NaN,Yes,No


In [10]:
df2 = get_evidence_df('/work/frink/mcinerney.de/ehr-diagnosis-annotations/annotations_9-7-23/annotations/val2/Andrea_Young/')
df2[1:]

,info,obs,timestep,num_reports,instance,seen_targets,option_likelihood_anns,prediction_aligns,evidence_anns,options,selected_options,sort_by_model_order,option_likelihood_anns2,concluding_notes
1,"{'max_timesteps': 10, 'current_targets': {'pne...","{'options': 'option,type cancer,diagnosis pneu...",1,35,"val2 Instance 45 (patient 28898, 113 reports)",{},"{'cancer (diagnosis)': 'Unlikely', 'pneumonia ...",Yes,{'LLM Confidence': {34: {'relevance': '3 - Ver...,"[cancer (diagnosis), pneumonia (diagnosis)]","[0, 1]",[LLM Confidence],"{'cancer (diagnosis)': 'Unlikely', 'pneumonia ...",
2,"{'max_timesteps': 2, 'current_targets': {'pulm...","{'options': 'option,type cancer,diagnosis pneu...",1,1,"val2 Instance 46 (patient 15170, 186 reports)",{},"{'cancer (diagnosis)': 'Unlikely', 'pneumonia ...",No,{'Sort by MSE with the bias logits': {0: {'rel...,"[cancer (diagnosis), pneumonia (diagnosis), pu...","[0, 1, 2]",[Sort by MSE with the bias logits],"{'cancer (diagnosis)': 'Unlikely', 'pneumonia ...",
3,"{'max_timesteps': 12, 'current_targets': {'pne...","{'options': 'option,type cancer,diagnosis pneu...",1,52,"val2 Instance 50 (patient 11755, 122 reports)",{},"{'cancer (diagnosis)': 'Unlikely', 'pneumonia ...",Yes,{'Sort by MSE with the bias logits': {1: {'rel...,"[cancer (diagnosis), pneumonia (diagnosis)]","[0, 1]",[Sort by MSE with the bias logits],"{'cancer (diagnosis)': 'Unlikely', 'pneumonia ...",


In [11]:
df2.iloc[1]

info                       {'max_timesteps': 10, 'current_targets': {'pne...
obs                        {'options': 'option,type
cancer,diagnosis
pneu...
timestep                                                                   1
num_reports                                                               35
instance                       val2 Instance 45 (patient 28898, 113 reports)
seen_targets                                                              {}
option_likelihood_anns     {'cancer (diagnosis)': 'Unlikely', 'pneumonia ...
prediction_aligns                                                        Yes
evidence_anns              {'LLM Confidence': {34: {'relevance': '3 - Ver...
options                          [cancer (diagnosis), pneumonia (diagnosis)]
selected_options                                                      [0, 1]
sort_by_model_order                                         [LLM Confidence]
option_likelihood_anns2    {'cancer (diagnosis)': 'Unlikely', 'pneumonia ...